In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern, WhiteKernel
from sklearn import linear_model
from sklearn.metrics import log_loss

from scipy.stats import zscore, norm

import itertools

np.random.seed(1)

# Variables that the model will be trained on (and the Team Name)
TO_KEEP = ['Adj Off Efficiency', 'Adj Off-Def', 'Adj Def Efficiency', 'FG%',
           'Strength of Schedule', 'Avg. Scoring margin', 'Rebounds', 'Wins Last 10 Games ',
           'Turnovers per game', 'Coach Record']

def train_on_tk(to_keep):
    # Training data
    combo = pd.read_csv('2017-2018_combo.csv')
    df = pd.read_excel('cts.xlsx', sheet_name='2017-2018')
    df = df[to_keep]
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].apply(zscore)

    # Validation data
    coval = pd.read_csv('2016-2017_combo.csv')
    val = pd.read_excel('cts.xlsx', sheet_name='2016-2017')
    val = val[to_keep]
    numeric_cols = val.select_dtypes(include=[np.number]).columns
    val[numeric_cols] = val[numeric_cols].apply(zscore)

    ########################################
    
    Xs = []
    ys = []

    for idx, row in combo.iterrows():
        winner = row['Winner']
        loser = row['Loser']

        w_v = np.array(df[df['Team Name'] == winner])[0][1:]
        l_v = np.array(df[df['Team Name'] == loser])[0][1:]

        Xs.append(np.array([w_v, l_v]).flatten())
        ys.append(np.array([row['Winner Points'] - row['Loser points']]))

        Xs.append(np.array([l_v, w_v]).flatten())
        ys.append(np.array([row['Loser points'] - row['Winner Points']]))

    X = np.array(Xs)
    y = np.array(ys)

    Xvs = []
    yvs = []

    for idx, row in coval.iterrows():
        winner = row['Winner']
        loser = row['Loser']

        try:
            w_v = np.array(val[val['Team Name'] == winner])[0][1:]
            l_v = np.array(val[val['Team Name'] == loser])[0][1:]
            Xvs.append(np.array([w_v, l_v]).flatten())
            yvs.append(np.array([row['Winner Points'] - row['Loser points']]))

            Xvs.append(np.array([l_v, w_v]).flatten())
            yvs.append(np.array([row['Loser points'] - row['Winner Points']]))
        except:
            print(winner)
            print(loser)

    Xv = np.array(Xvs)
    yv = np.array(yvs)

    ########################################

    rep = -1000
    rbest_ = 0
    obest_ = 0
    wbest_ = 0

    rbest_ = 54.5618245614035
    obest_ = 18.538748538011692
    wbest_ = 84.34343436464646

    # rbest = 57.89515789473683
    # obest = 26.31652631578947
    # wbest = 78.7878788090909

    # print('starting')
    # count=0
    # for rbf_val in np.linspace(rbest - 10, rbest + 10, 10):
    #     for outside in np.linspace(obest - 10, obest + 10, 10):
    #         for wkb in np.linspace(wbest - 10, wbest + 10, 10):
    #             kernel = outside * RBF(rbf_val) + WhiteKernel(wkb)    
    #             gp = GaussianProcessRegressor(kernel,
    #                                           n_restarts_optimizer=10,
    #                                           normalize_y=True)
    #             gp.fit(X, y)
    #             score = gp.score(Xv, yv)
    #             if score > rep:
    #                 rbest_ = rbf_val
    #                 obest_ = outside
    #                 wbest_ = wkb
    #                 rep = score

    #             count += 1
    #             if count % int(10*10*10/100) == 0:
    #                 print(f'{count / (10*10*10)}')

    # print('GP Validation R^2: ' + str(rep))
    # print(f'Best RBF Length Scale: {rbest_}')
    # print(f'Best RBF Kernel Scale: {obest_}')
    # print(f'Best White Noise: {wbest_}')
    ########################################

    kernel = obest_ * RBF(rbest_) + WhiteKernel(wbest_)
    gp = GaussianProcessRegressor(kernel,
                                      n_restarts_optimizer=10,
                                      normalize_y=True)
    gp.fit(X, y)
    rep = gp.score(Xv, yv)

    print('GP Validation R^2: ' + str(rep))

    ########################################

    lm = linear_model.LinearRegression(fit_intercept=False, normalize=True)
    lm.fit(X, y)
    lm.score(Xv, yv)

    ########################################

    gp_y_pred = []
    lm_y_pred = []
    gp_sd = []
    tys = []
    for idx, row in coval.iterrows():
        winner = row['Winner']
        loser = row['Loser']

        try:
            w_v = np.array(val[val['Team Name'] == winner])[0][1:]
            l_v = np.array(val[val['Team Name'] == loser])[0][1:]
            Xvs.append(np.array([w_v, l_v]).flatten())
            yvs.append(np.array([row['Winner Points'] - row['Loser points']]))

            gp_y_pred_, gp_sd_ = gp.predict(np.array([w_v, l_v]).flatten().reshape(1, -1), return_std=True)
            gp_y_pred.append(gp_y_pred_)
            gp_sd.append(gp_sd_)
            lm_y_pred.append(lm.predict(np.array([w_v, l_v]).flatten().reshape(1, -1)))
            tys.append(row['Winner Points'] - row['Loser points'])
        except:
            print(winner)
            print(loser)

    gp_y_pred=np.array(gp_y_pred).flatten()
    lm_y_pred=np.array(lm_y_pred).flatten()
    gp_sd = np.array(gp_sd).flatten()
    tys = np.array(tys).flatten()

    ########################################

    correct = 0
    for i in range(len(tys)):
        if tys[i] >= 0:
            if lm_y_pred[i] >= 0:
                correct += 1
        else:
            if lm_y_pred[i] < 0:
                correct += 1

    print('Percent of Games Predicted Correctly by LM: ' + str(correct/len(tys)))

    correct = 0
    for i in range(len(tys)):
        if tys[i] >= 0:
            if gp_y_pred[i] >= 0:
                correct += 1
        else:
            if gp_y_pred[i] < 0:
                correct += 1

    print('Percent of Games Predicted Correctly by GP: ' + str(correct / len(tys)))

    y_pred_probs = 1 - norm.cdf(-gp_y_pred / gp_sd)
    ll = log_loss(np.array(tys > 0, dtype=np.int), np.array([1 - y_pred_probs, y_pred_probs]).T, labels=[0, 1])
    print('GP Log-Loss: ' + str(ll))
    
    return ll

In [25]:
max_ss = None
max_ll = 1
for i in range(1, len(TO_KEEP)):
    for to_keep in set(itertools.combinations(TO_KEEP, i)):
        ll = train_on_tk(['Team Name'] + list(to_keep))
        if ll < max_ll:
            max_ll = ll
            max_ss = to_keep

GP Validation R^2: -1.3347574157052122e-09
Percent of Games Predicted Correctly by LM: 0.5714285714285714
Percent of Games Predicted Correctly by GP: 0.4126984126984127
GP Log-Loss: 0.6931471820089533
GP Validation R^2: -0.0008316657764728675
Percent of Games Predicted Correctly by LM: 0.6825396825396826
Percent of Games Predicted Correctly by GP: 0.9523809523809523
GP Log-Loss: 0.694339165217516
GP Validation R^2: 0.0
Percent of Games Predicted Correctly by LM: 0.6507936507936508
Percent of Games Predicted Correctly by GP: 1.0
GP Log-Loss: 0.6931471805599455
GP Validation R^2: 0.31539716035572174
Percent of Games Predicted Correctly by LM: 0.7619047619047619
Percent of Games Predicted Correctly by GP: 0.7619047619047619
GP Log-Loss: 0.5468173304263724
GP Validation R^2: 0.2955598193630926
Percent of Games Predicted Correctly by LM: 0.7142857142857143
Percent of Games Predicted Correctly by GP: 0.7142857142857143
GP Log-Loss: 0.5940048822852914
GP Validation R^2: 0.23028556280212753
Pe

GP Validation R^2: 0.10714083777871775
Percent of Games Predicted Correctly by LM: 0.7142857142857143
Percent of Games Predicted Correctly by GP: 0.6507936507936508
GP Log-Loss: 0.6626361774856253
GP Validation R^2: 0.30292372527633593
Percent of Games Predicted Correctly by LM: 0.6825396825396826
Percent of Games Predicted Correctly by GP: 0.6984126984126984
GP Log-Loss: 0.5825778733838899
GP Validation R^2: 0.03788088794075173
Percent of Games Predicted Correctly by LM: 0.5873015873015873
Percent of Games Predicted Correctly by GP: 0.5714285714285714
GP Log-Loss: 0.6788470654383862
GP Validation R^2: -0.4477402788852005
Percent of Games Predicted Correctly by LM: 0.4126984126984127
Percent of Games Predicted Correctly by GP: 0.4126984126984127
GP Log-Loss: 0.8564978106834471
GP Validation R^2: 0.26361454452348543
Percent of Games Predicted Correctly by LM: 0.6825396825396826
Percent of Games Predicted Correctly by GP: 0.7142857142857143
GP Log-Loss: 0.6079543939709463
GP Validation R

GP Validation R^2: 0.2212670584224803
Percent of Games Predicted Correctly by LM: 0.6666666666666666
Percent of Games Predicted Correctly by GP: 0.6666666666666666
GP Log-Loss: 0.6091874835927567
GP Validation R^2: 0.006482955377489552
Percent of Games Predicted Correctly by LM: 0.3968253968253968
Percent of Games Predicted Correctly by GP: 0.5714285714285714
GP Log-Loss: 0.6823617769315091
GP Validation R^2: 0.1374549641859435
Percent of Games Predicted Correctly by LM: 0.7142857142857143
Percent of Games Predicted Correctly by GP: 0.6190476190476191
GP Log-Loss: 0.6281616727824944
GP Validation R^2: 0.25133984118021413
Percent of Games Predicted Correctly by LM: 0.6507936507936508
Percent of Games Predicted Correctly by GP: 0.7142857142857143
GP Log-Loss: 0.6109882888059074
GP Validation R^2: 0.25770426600097707
Percent of Games Predicted Correctly by LM: 0.6984126984126984
Percent of Games Predicted Correctly by GP: 0.6825396825396826
GP Log-Loss: 0.5896785806340566
GP Validation R^

GP Validation R^2: 4.0801502365628295e-07
Percent of Games Predicted Correctly by LM: 0.6825396825396826
Percent of Games Predicted Correctly by GP: 0.5238095238095238
GP Log-Loss: 0.6931463672743288
GP Validation R^2: 0.2756660932166891
Percent of Games Predicted Correctly by LM: 0.6666666666666666
Percent of Games Predicted Correctly by GP: 0.6825396825396826
GP Log-Loss: 0.5971498800284402
GP Validation R^2: 0.37161731874319465
Percent of Games Predicted Correctly by LM: 0.7301587301587301
Percent of Games Predicted Correctly by GP: 0.7142857142857143
GP Log-Loss: 0.5281418144691077
GP Validation R^2: 8.637739235706209e-05
Percent of Games Predicted Correctly by LM: 0.746031746031746
Percent of Games Predicted Correctly by GP: 0.6190476190476191
GP Log-Loss: 0.6931569676856467
GP Validation R^2: 0.2438458971881986
Percent of Games Predicted Correctly by LM: 0.6666666666666666
Percent of Games Predicted Correctly by GP: 0.6349206349206349
GP Log-Loss: 0.5916178355507016
GP Validation

GP Validation R^2: 0.39188309620591216
Percent of Games Predicted Correctly by LM: 0.7142857142857143
Percent of Games Predicted Correctly by GP: 0.6984126984126984
GP Log-Loss: 0.539634170315071
GP Validation R^2: 0.4146210639679788
Percent of Games Predicted Correctly by LM: 0.7936507936507936
Percent of Games Predicted Correctly by GP: 0.7936507936507936
GP Log-Loss: 0.5101950607382393
GP Validation R^2: -0.4386897977665136
Percent of Games Predicted Correctly by LM: 0.3333333333333333
Percent of Games Predicted Correctly by GP: 0.36507936507936506
GP Log-Loss: 0.8545473278388829
GP Validation R^2: -0.12473407361976485
Percent of Games Predicted Correctly by LM: 0.47619047619047616
Percent of Games Predicted Correctly by GP: 0.49206349206349204
GP Log-Loss: 0.7203567889471761
GP Validation R^2: -0.30112842206152846
Percent of Games Predicted Correctly by LM: 0.4126984126984127
Percent of Games Predicted Correctly by GP: 0.42857142857142855
GP Log-Loss: 0.798365974778126
GP Validatio

...

GP Validation R^2: 0.33205101274668586
Percent of Games Predicted Correctly by LM: 0.6984126984126984
Percent of Games Predicted Correctly by GP: 0.6825396825396826
GP Log-Loss: 0.5516243905428436
GP Validation R^2: 0.24554282764066881
Percent of Games Predicted Correctly by LM: 0.6984126984126984
Percent of Games Predicted Correctly by GP: 0.7301587301587301
GP Log-Loss: 0.5680995496867453
GP Validation R^2: 0.3371171519871136
Percent of Games Predicted Correctly by LM: 0.6825396825396826
Percent of Games Predicted Correctly by GP: 0.7301587301587301
GP Log-Loss: 0.5167904663594625
GP Validation R^2: 0.17037104459124375
Percent of Games Predicted Correctly by LM: 0.6825396825396826
Percent of Games Predicted Correctly by GP: 0.7301587301587301
GP Log-Loss: 0.5861835335654113
GP Validation R^2: 0.3586404493844617
Percent of Games Predicted Correctly by LM: 0.6349206349206349
Percent of Games Predicted Correctly by GP: 0.7936507936507936
GP Log-Loss: 0.5080412472458438
GP Validation R^2

GP Validation R^2: 0.14051737668083408
Percent of Games Predicted Correctly by LM: 0.6984126984126984
Percent of Games Predicted Correctly by GP: 0.6825396825396826
GP Log-Loss: 0.6273525721729534
GP Validation R^2: 0.26958814548790544
Percent of Games Predicted Correctly by LM: 0.6507936507936508
Percent of Games Predicted Correctly by GP: 0.7142857142857143
GP Log-Loss: 0.5651324070530471
GP Validation R^2: 0.19341181474226776
Percent of Games Predicted Correctly by LM: 0.6190476190476191
Percent of Games Predicted Correctly by GP: 0.746031746031746
GP Log-Loss: 0.5817242623747833
GP Validation R^2: 0.15732274198886775
Percent of Games Predicted Correctly by LM: 0.6349206349206349
Percent of Games Predicted Correctly by GP: 0.6984126984126984
GP Log-Loss: 0.5958343764324457
GP Validation R^2: 0.22275801977315943
Percent of Games Predicted Correctly by LM: 0.5555555555555556
Percent of Games Predicted Correctly by GP: 0.746031746031746
GP Log-Loss: 0.590319663288207
GP Validation R^2:

GP Validation R^2: 0.18422712525840756
Percent of Games Predicted Correctly by LM: 0.6825396825396826
Percent of Games Predicted Correctly by GP: 0.6984126984126984
GP Log-Loss: 0.6100135165274126
GP Validation R^2: 0.18136309950438523
Percent of Games Predicted Correctly by LM: 0.5714285714285714
Percent of Games Predicted Correctly by GP: 0.6507936507936508
GP Log-Loss: 0.6079088416212585
GP Validation R^2: 0.38425710497504023
Percent of Games Predicted Correctly by LM: 0.6190476190476191
Percent of Games Predicted Correctly by GP: 0.746031746031746
GP Log-Loss: 0.5191276934965368
GP Validation R^2: 0.34192919603120214
Percent of Games Predicted Correctly by LM: 0.6825396825396826
Percent of Games Predicted Correctly by GP: 0.7301587301587301
GP Log-Loss: 0.5512205564645075
GP Validation R^2: 0.3697004883008398
Percent of Games Predicted Correctly by LM: 0.6666666666666666
Percent of Games Predicted Correctly by GP: 0.7777777777777778
GP Log-Loss: 0.526559759617961
GP Validation R^2:

GP Validation R^2: 0.06526139597675062
Percent of Games Predicted Correctly by LM: 0.5238095238095238
Percent of Games Predicted Correctly by GP: 0.5873015873015873
GP Log-Loss: 0.6586218875521382
GP Validation R^2: 0.18377956329618983
Percent of Games Predicted Correctly by LM: 0.6984126984126984
Percent of Games Predicted Correctly by GP: 0.7142857142857143
GP Log-Loss: 0.6112379620365982
GP Validation R^2: 0.18418344293327737
Percent of Games Predicted Correctly by LM: 0.6349206349206349
Percent of Games Predicted Correctly by GP: 0.6984126984126984
GP Log-Loss: 0.5831959264997008
GP Validation R^2: 0.1880236312479403
Percent of Games Predicted Correctly by LM: 0.6984126984126984
Percent of Games Predicted Correctly by GP: 0.7301587301587301
GP Log-Loss: 0.5921028618992954
GP Validation R^2: 0.19990211030418484
Percent of Games Predicted Correctly by LM: 0.6507936507936508
Percent of Games Predicted Correctly by GP: 0.7301587301587301
GP Log-Loss: 0.5802887215122938
GP Validation R^

GP Validation R^2: 0.2155155573836003
Percent of Games Predicted Correctly by LM: 0.5714285714285714
Percent of Games Predicted Correctly by GP: 0.7142857142857143
GP Log-Loss: 0.5781263894617941
GP Validation R^2: 0.3768343769630068
Percent of Games Predicted Correctly by LM: 0.6507936507936508
Percent of Games Predicted Correctly by GP: 0.7936507936507936
GP Log-Loss: 0.5032847742563896
GP Validation R^2: 0.12197642435893752
Percent of Games Predicted Correctly by LM: 0.6349206349206349
Percent of Games Predicted Correctly by GP: 0.6825396825396826
GP Log-Loss: 0.6258726337512523
GP Validation R^2: 0.18277363989567386
Percent of Games Predicted Correctly by LM: 0.7142857142857143
Percent of Games Predicted Correctly by GP: 0.746031746031746
GP Log-Loss: 0.5943692898105964
GP Validation R^2: 0.17686110110453257
Percent of Games Predicted Correctly by LM: 0.6666666666666666
Percent of Games Predicted Correctly by GP: 0.7142857142857143
GP Log-Loss: 0.5880834056895252
GP Validation R^2:

In [26]:
max_ll

0.46736130156443045

In [27]:
max_ss

('Adj Off Efficiency',
 'Adj Def Efficiency',
 'Wins Last 10 Games ',
 'Turnovers per game')